In [1]:
import pickle
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.activations import relu,softmax

2023-03-27 19:30:43.329190: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 19:30:43.514455: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-27 19:30:43.517788: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 19:30:43.517807: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
x=pickle.load(open('data.pkl','rb'))
y=pickle.load(open('data_label.pkl','rb'))
output_neurons=len(set(y))

In [3]:
output_neuron=len(set(y))

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15)

In [5]:
input_shape=x_train.shape[1]


In [6]:
def create_model(layers,activation):
    model=Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
             model.add(Dense(128,input_shape=(input_shape,)))
        model.add(Activation(activation))
        model.add(Dropout(0.5))
        model.add(Dense(nodes))
        model.add(Activation(activation))
        model.add(Dropout(0.5))
    model.add(Dense(output_neuron))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])#sparse categorical crossentropy works on integer targets
    return model

In [7]:
from keras.wrappers.scikit_learn import KerasClassifier
model=KerasClassifier(build_fn=create_model)

/tmp/ipykernel_92236/649555917.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model)


In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
layers=[[100],[120,90],[120,100,90]]
activations=['sigmoid','relu']
param_grid=dict(layers=layers,activation=activations,batch_size=[30,50],epochs=[100,200,75])


In [10]:
clf=GridSearchCV(estimator= model,param_grid=param_grid,cv=5,scoring='accuracy')
clf.fit(x_train,y_train,verbose=0)

2023-03-27 19:31:20.530596: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-27 19:31:20.530644: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-27 19:31:20.530670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (shiva1974-VivoBook-ASUSLaptop-X515JA-X515JA): /proc/driver/nvidia/version does not exist
2023-03-27 19:31:20.530978: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 76ms/step


GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f4d19edb9a0>,
             param_grid={'activation': ['sigmoid', 'relu'],
                         'batch_size': [30, 50], 'epochs': [100, 200, 75],
                         'layers': [[100], [120, 90], [120, 100, 90]]},
             scoring='accuracy')

In [11]:
clf.best_params_

{'activation': 'relu', 'batch_size': 50, 'epochs': 200, 'layers': [100]}

In [12]:
best_model=create_model([100],'relu')

In [13]:
history=best_model.fit(x_train,y_train,epochs=200)

Epoch 1/200
4/4 [==============================] - 1s 4ms/step - loss: 3.1138 - accuracy: 0.0413
Epoch 2/200
4/4 [==============================] - 0s 4ms/step - loss: 3.0647 - accuracy: 0.0826
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 3.0503 - accuracy: 0.0826
Epoch 4/200
4/4 [==============================] - 0s 4ms/step - loss: 3.0264 - accuracy: 0.1074
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 3.0002 - accuracy: 0.1157
Epoch 6/200
4/4 [==============================] - 0s 4ms/step - loss: 2.9600 - accuracy: 0.1736
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 2.9362 - accuracy: 0.1157
Epoch 8/200
4/4 [==============================] - 0s 4ms/step - loss: 2.9112 - accuracy: 0.1653
Epoch 9/200
4/4 [==============================] - 0s 4ms/step - loss: 2.8444 - accuracy: 0.2397
Epoch 10/200
4/4 [==============================] - 0s 4ms/step - loss: 2.8321 - accuracy: 0.2397
Epoch 11/200
4/4 [===========

In [14]:
best_model.evaluate(x_test,y_test)

1/1 [==============================] - 0s 141ms/step - loss: 0.3514 - accuracy: 0.9091


[0.35142388939857483, 0.9090909361839294]

In [19]:
best_model.save('chatbot_model.h5')

In [ ]:
pickle.dump(best_model,open('chatbot_model.pkl','wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\activation
......vars
...layers\activation_1
......vars
...layers\activation_2
......vars
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-02-10 21:01:38         2559
metadata.json                                  2023-02-10 21:01:38           64
variables.h5                                   2023-02-10 21:01: